### Web scrapping using Beautifulsoup and Selenium
In this notebook, we scape the ministry list from different links on http://goidirectory.nic.in/sector_index.php

In [1]:
import sys
import webbrowser
import requests
from bs4 import BeautifulSoup as BS
import codecs
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# get the path of ChromeDriverServer

chrome_driver_path =  "mypath\Anaconda2\selenium\chromedriver.exe"
reload(sys)  
sys.setdefaultencoding('utf8')

In [2]:
def open_page(link,main_group):
    driver = webdriver.Chrome(chrome_driver_path)
    driver.implicitly_wait(30)
    driver.maximize_window()
    driver.get(link)
    lvl1 =main_group
    xpath='//*[@title="'+lvl1+'"]'
    main_click = driver.find_element_by_xpath(xpath).click()
    link = driver.current_url
    driver.close()
    return link

In [3]:
def scrap(link,q):
    # For the main page
    if q==1:
        html = requests.get(link[0])
        text = html.text
        soup = BS(text,"html.parser") 
        main_links=[]
        for i in soup.find_all("a",{'class' : "heading_light_url"}):
            main_links.append(i.text)
        return main_links  
    # For sub pages
    else:
        all_min=[]
        for i in link.find_all("a",{'class' : "heading_light_url"}):
            all_min.append(i.text)
        return all_min    
        

In [4]:
# Scrap the main sectors on 1st page
main_links=scrap(["http://goidirectory.nic.in/sector_index.php"],1)
# Removing the numbers at the end
for i,val in enumerate(main_links):
    main_links[i]=main_links[i][:main_links[i].index(' (')]    

In [102]:
ministry={}
for i in main_links:
    main_link = open_page("http://goidirectory.nic.in/sector_index.php",i)
    s= open_A_Z_page(main_link)
    ministry[i]=scrap(s,2)

In [104]:
ministry

{u'Agriculture & Cooperation': [u'Agricultural Census, New Delhi',
  u'Agricultural Engineering Department',
  u'Agricultural Finance Corporation Limited (AFCL), Mumbai',
  u'Agricultural Marketing Department',
  u'Agricultural Marketing Department, Karnataka',
  u'Agricultural Marketing Department, Telangana',
  u'Agricultural Meteorology Division',
  u'Agricultural Planning and Information Bank(APIB), East Khasi Hills District, Meghalaya',
  u'Agricultural Resources Information System (AgRIS)',
  u'Agricultural Scientists Recruitment Board (ASRB), New Delhi',
  u'Agricultural Technology Information Centre (ATIC)',
  u'Agricultural Technology Management Agency (ATMA), Government of Jammu and Kashmir',
  u'Agricultural Technology Management Agency (ATMA), Kohima, Nagaland',
  u'Agriculture and Cooperation Department, Gujarat',
  u'Agriculture, Animal Husbandry and Co-operative Department, Jharkhand',
  u'Agriculture Department',
  u'Agriculture Department, Andaman and Nicobar',
  u'Agr

#### Convert to Pandas dataframe

In [107]:
df = pd.DataFrame(ministry.items(),columns=['Group','Ministry'])

In [108]:
s=df.apply(lambda x: pd.Series(x['Ministry']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Min'

In [109]:
x=df.drop('Ministry', axis=1).join(s)

In [110]:
x.to_csv('Ministry List.csv')